In [1]:
!pip install transformers datasets torch
!pip install unsloth
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
from huggingface_hub import login
from unsloth import FastLanguageModel
from peft import LoraConfig, get_peft_model

<ipython-input-2-ab99e013b56e>:6: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Log in to Hugging Face Hub to save the fine-tuned model
login()

In [ ]:
from datasets import Dataset

# Dataset: Incorrect math memes and their corrected versions
dataset = [
    {"riddle": "8 ÷ 2(2+2) = 1?", "solution": "8 ÷ 2 × (4) = 4 × 4 = 16"},
    {"riddle": "52 = 10?", "solution": "5² = 25"},
    {"riddle": "6 ÷ 2(1+2) = 1?", "solution": "6 ÷ 2 × (3) = 3 × 3 = 9"},
    {"riddle": "0.999... = 0?", "solution": "0.999... = 1 (Mathematical proof: Limit approach)"},
    {"riddle": "2 + 3 × 4 = 20?", "solution": "2 + (3 × 4) = 2 + 12 = 14"},
    {"riddle": "100 - 50 ÷ 2 = 25?", "solution": "100 - (50 ÷ 2) = 100 - 25 = 75"},
    {"riddle": "1 + 1 = 3 (common meme)?", "solution": "1 + 1 = 2 (Basic arithmetic)"},
    {"riddle": "10 ÷ 5 ÷ 2 = 1?", "solution": "(10 ÷ 5) ÷ 2 = 2 ÷ 2 = 1"},
    {"riddle": "25% of 200 = 100?", "solution": "25% of 200 = (25/100) × 200 = 50"},
    {"riddle": "1/2 + 1/2 = 1/4?", "solution": "1/2 + 1/2 = 1"},
    {"riddle": "√-1 = 1?", "solution": "√-1 = i (Imaginary unit in complex numbers)"},
    {"riddle": "10² = 20?", "solution": "10² = 100"},
    {"riddle": "9 - 3 ÷ 1/3 + 1 = 1?", "solution": "9 - 3 ÷ (1/3) + 1 = 9 - 9 + 1 = 1"},
    {"riddle": "2⁴ ÷ 2² = 8?", "solution": "2⁴ ÷ 2² = 16 ÷ 4 = 4"},
    {"riddle": "3(2+2) = 3×2+2?", "solution": "3(2+2) = 3×4 = 12 (not 3×2+2 = 6+2 = 8)"},
    {"riddle": "7 + 7 ÷ 7 + 7 × 7 - 7 = 0?", "solution": "7 + (7 ÷ 7) + (7 × 7) - 7 = 7 + 1 + 49 - 7 = 50"},
    {"riddle": "4 ÷ 2(2+2) = 2?", "solution": "4 ÷ 2 × (4) = 2 × 4 = 8"},
    {"riddle": "50 - 10 × 5 + 10 = 200?", "solution": "50 - (10 × 5) + 10 = 50 - 50 + 10 = 10"},
    {"riddle": "√4 + √4 = 4?", "solution": "√4 + √4 = 2 + 2 = 4"},
    {"riddle": "60 ÷ 5(7 - 5) = 30?", "solution": "60 ÷ 5 × (7 - 5) = 60 ÷ 5 × 2 = 12 × 2 = 24"},
]

# Convert the dataset into a Hugging Face Dataset
dataset = Dataset.from_dict({
    "riddle": [item["riddle"] for item in dataset],
    "solution": [item["solution"] for item in dataset]
})


In [ ]:

# Load the pre-trained model and tokenizer with Unsloth
model_name = "deepseek-ai/deepseek-math-7b-instruct"  # Replace with the correct model name
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=512,
    dtype=None,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

deepseek-ai/deepseek-math-7b-instruct does not have a padding token! Will use pad_token = <|PAD_TOKEN|>.


In [ ]:
# Tokenize the riddle dataset
def preprocess_function(examples):
    # Tokenize the riddles (inputs)
    model_inputs = tokenizer(examples["riddle"], truncation=True, padding="max_length", max_length=512)

    # Tokenize the solutions (labels)
    labels = tokenizer(examples["solution"], truncation=True, padding="max_length", max_length=512)

    # Add the labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = more_dataset.map(preprocess_function, batched=True)

# Check the first example
print(tokenized_dataset[0])

# Decode the input IDs to verify the tokenization
decoded_input = tokenizer.decode(tokenized_dataset[0]["input_ids"])
decoded_labels = tokenizer.decode(tokenized_dataset[0]["labels"])

print(f"Decoded input: {decoded_input}")
print(f"Decoded labels: {decoded_labels}")


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

{'riddle': 'I’m tall when I’m young, and I’m short when I’m old. What am I?', 'solution': 'A candle', 'solution_explanation': 'A candle burns down as it gets older, becoming shorter.', 'input_ids': [100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 100002, 1

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=500,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,  # Enable mixed precision training to save memory
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=50,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=500,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,  # Enable mixed precision training to save memory
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
!pip install peft

In [ ]:
# Define LoRA configuration for PEFT
lora_config = LoraConfig(
    r=16,  # Rank of the low-rank adaptation
    lora_alpha=32,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Target modules for LoRA
    lora_dropout=0.05,  # Dropout for LoRA
    bias="none",  # No bias
    task_type="SEQ_2_SEQ_LM",  # Task type for sequence-to-sequence models
)

# Wrap the model with PEFT
model = get_peft_model(model, lora_config)

In [ ]:
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 15 | Num Epochs = 50 | Total steps = 400
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 7,864,320/7,000,000,000 (0.11% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: f219054 (f219054-national-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,6.138974
2,18.830900,6.088425


KeyboardInterrupt: 

In [ ]:
# Save the model and tokenizer locally
model.save_pretrained("./fine-tuned-deepseek-math")
tokenizer.save_pretrained("./fine-tuned-deepseek-math")

('./fine-tuned-deepseek-math/tokenizer_config.json',
 './fine-tuned-deepseek-math/special_tokens_map.json',
 './fine-tuned-deepseek-math/tokenizer.json')

In [ ]:
prompt = "Riddle: What number becomes zero when you subtract 15 from half of it ?\nSolution:"

In [ ]:
# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt")  # `return_tensors="pt"` returns PyTorch tensors

In [ ]:
# Move inputs to the same device as the model
inputs = {key: value.to("cuda") for key, value in inputs.items()}

In [ ]:
# Generate a response
outputs = model.generate(**inputs, max_length=50)  # `max_length` controls the maximum length of the output

In [ ]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Riddle: What number becomes zero when you subtract 15 from half of it ?
Solution: Let the number be x.
According to the question,
x/2 - 15 = 0
=> x/2
